In [3]:
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np

import GCode
import GCode.Line
import GRBL


# Code:

Draw a 10 mm line from (0, 0) to (10, 0).

In [4]:
progs = list()

for n_points in range(2, 10):
    
    line_cfg = {
        "X0": 0,
        "Xf": 10,
        "Y": 0,
        "n_points": n_points
    }    
    points = GCode.HorzLine(**line_cfg)
    
    line_cfg = {
        "points": points,
        "feed":120,
        "power":128,
        "dynamic_power": True,
    }
    line = GCode.HorzLine(points=points)
    
    
    prog_cfg={
        "lines": [line, line],
        "feed": 120
    }
    prog = GCode.HorzLine(**prog_cfg)
    progs.append(prog)
progs

AttributeError: type object 'Line' has no attribute 'HorzLine'

In [ ]:
for prog in progs:
    print(len(prog.buffer))

In [ ]:
for prog in progs:
    prog.generate_gcode()
    print(len(prog.buffer))

In [ ]:
list(map(lambda prog: prog.generate_gcode(), progs))


In [ ]:
list(map(lambda prog: len(prog.buffer), progs))

In [ ]:
import threading

def concurrent_map(func, data):
    """
    Similar to the bultin function map(). But spawn a thread for each argument
    and apply `func` concurrently.

    Note: unlike map(), we cannot take an iterable argument. `data` should be an
    indexable sequence.
    """

    N = len(data)
    result = [None] * N

    # wrapper to dispose the result in the right slot
    def task_wrapper(i):
        result[i] = func(data[i])

    threads = [threading.Thread(target=task_wrapper, args=(i,)) for i in range(N)]
    for t in threads:
        t.start()
    for t in threads:
        t.join()

    return result

In [ ]:
concurrent_map(lambda prog: prog.generate_gcode(), progs)

In [ ]:
concurrent_map(lambda prog: len(prog.buffer), progs)

In [ ]:
concurrent_map(lambda prog: prog.__repr__(), progs)

In [ ]:
concurrent_map(lambda prog: prog.dist, progs)

In [ ]:
concurrent_map(lambda prog: prog.jog_dist, progs)

In [ ]:
concurrent_map(lambda prog: prog.laserin_dist, progs)

In [ ]:
m=concurrent_map(lambda prog: prog.laserin_dist, progs)

In [ ]:
np.diff(m)

In [ ]:
np.diff(m)==0

In [ ]:
np.all(np.diff(m)==0)

In [ ]:
assert(np.all(np.diff(m)==0))

In [ ]:
flip2

In [ ]:
reverse(None, progs[1].lines[0].points)

In [ ]:
progs